# Plot dati scaricati da Justur
- dati esportati da db "measurements"
- Plot dati aggregati (medie) per ogni punto di taratura

Rev. 0

In [1]:
import pandas as pd
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter

## Importo dati da taratura justur
- Dati esportati da db "measurements"
- rimosso da csv Q_REG

In [2]:
df = pd.read_csv("DN50_GS500-20250917-corretti.csv", sep = ";",decimal=',')

In [17]:
# Aggrago dati per ID e punto taratura, calcolo media portata, media errori, devstd errore
df_agg = df.groupby(['ID','Ripetizione']).agg(
    Q_m=("Q","mean"),
    E_medio=("E", "mean"),
    Dev_std_E=("E", "std")
)

# --- df MultiIndex ---
# otteniamo colonne normali: ID, Ripetizione, Q_m, E_medio, Dev_std_E
df_agg= df_agg.reset_index() 

In [18]:
df_agg

ID  Ripetizione        Q_m    E_medio  Dev_std_E
0   52343            3  38.705667   0.041779   0.029182
1   52343            4   0.168184  -4.961637   0.473019
2   52343            5   0.104904  -8.901150   0.444924
3   52344            3  38.705667  -0.043060   0.035391
4   52344            4   0.168184  -8.763020   0.401940
5   52344            5   0.104904 -15.363567   0.336046
6   52345            3  38.705667  -0.040007   0.075674
7   52345            4   0.168184  -5.054653   0.087160
8   52345            5   0.104904  -8.083070   0.504098
9   52346            3  38.705667   0.040918   0.010062
10  52346            4   0.168086  -6.299490   0.592923
11  52346            5   0.104904  -9.667803   0.512625
12  52347            3  38.705667  -0.027507   0.022708
13  52347            4   0.168184  -9.898877   0.179257
14  52347            5   0.104904 -17.105467   0.804517

In [19]:
# Assicuriamoci dei tipi numerici (utile se i dati arrivano da CSV/excel)
for c in ['ID', 'Ripetizione', 'Q_m', 'E_medio', 'Dev_std_E']:
    df_agg[c] = pd.to_numeric(df_agg[c], errors='coerce')

In [21]:
# Opzionale: ordina i punti per tracciare linee nel verso desiderato.
# Qui ordino per Q_m decrescente (38.7 -> 0.168 -> 0.105) per ogni ID
df_agg = df_agg.sort_values(['ID', 'Q_m'], ascending=[True, False])

In [22]:
df_agg

ID  Ripetizione        Q_m    E_medio  Dev_std_E
0   52343            3  38.705667   0.041779   0.029182
1   52343            4   0.168184  -4.961637   0.473019
2   52343            5   0.104904  -8.901150   0.444924
3   52344            3  38.705667  -0.043060   0.035391
4   52344            4   0.168184  -8.763020   0.401940
5   52344            5   0.104904 -15.363567   0.336046
6   52345            3  38.705667  -0.040007   0.075674
7   52345            4   0.168184  -5.054653   0.087160
8   52345            5   0.104904  -8.083070   0.504098
9   52346            3  38.705667   0.040918   0.010062
10  52346            4   0.168086  -6.299490   0.592923
11  52346            5   0.104904  -9.667803   0.512625
12  52347            3  38.705667  -0.027507   0.022708
13  52347            4   0.168184  -9.898877   0.179257
14  52347            5   0.104904 -17.105467   0.804517

In [11]:
# Palette fissa per mantenere coerenza dei colori
ids = df['ID'].unique()
palette = sns.color_palette('tab10', n_colors=len(ids))
pal = dict(zip(ids, palette))

In [12]:
sns.set_theme(style="whitegrid", context="talk")

## Plot Grafico con Seaborn
- media per ciascun ID calibrazione
- barre di errore
- x logaritmico

In [23]:
fig, ax = plt.subplots(figsize=(14, 8))

# Linee + marker per ogni ID (senza barre di errore qui)
sns.lineplot(
    data=df_agg,
    x='Q_m', y='E_medio',
    hue='ID', style='ID',
    marker='o', dashes=False,
    linewidth=2, markersize=8,
    palette=pal,
    errorbar=None,  # Disabilito l'errore di seaborn: usiamo matplotlib sotto
    ax=ax
)

# Aggiungo le barre di errore (Â± Dev_std_E) con lo stesso colore della linea
for _id, g in df_agg.groupby('ID'):
    ax.errorbar(
        g['Q_m'], g['E_medio'],
        yerr=g['Dev_std_E'],
        fmt='none',
        ecolor=pal[_id],
        elinewidth=1.4, capsize=4, capthick=1.2
    )

# Etichette e layout
ax.set_title('MUT7000 DN50 MID - E_medio vs Q_m con barre di errore')
ax.set_xlabel('Q [m3/h]', fontsize=15, labelpad=15)
ax.set_ylabel('E [%]', fontsize=15, labelpad=15)
#ax.legend(title='ID', bbox_to_anchor=(1.02, 1), loc='upper left', fontsize=12)
ax.legend(title='Cal ID', loc='lower right', fontsize=12)

# Personalizzazione assi
ax.set_xscale('log') # Imposta la scala logaritmica sull'asse X
# Applica ScalarFormatter per evitare la notazione scientifica
ax.xaxis.set_major_formatter(ScalarFormatter())
ax.xaxis.get_major_formatter().set_scientific(False)
ax.xaxis.get_major_formatter().set_useOffset(False)

ax.axhline(y=0, color='grey', linewidth=0.8) # Disegna asse a y=0
ax.grid(True, which='both', linestyle='--', linewidth=0.8)

plt.tight_layout()
plt.savefig("grafico2.png", dpi=300, bbox_inches='tight')
plt.show()